# First Neurons

In [ ]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

<b>Set Random Seeds for same results</b>

In [ ]:
np.random.seed(101)
tf.set_random_seed(101)

#### The graph we will be building

Getting a very simple linear fit to some 2d data.

<br/>
<div>
  <img src="./graph_for_Jose_06-29.jpg"
       alt="The TensorFlow Graph which we will be building for section 6-29."
       width="500px">
</div>
<br/>

<b>Data Setup</b>

Setting Up some Random Data for Demonstration Purposes

In [ ]:
rand_a = np.random.uniform(0, 100, (5, 5))
rand_a

In [ ]:
rand_b = np.random.uniform(0, 100, (5, 1))
rand_b

### Placeholders

In [ ]:
a = tf.placeholder(tf.float32)

In [ ]:
b = tf.placeholder(tf.float32) # we're not using the shape parameter for now.

### Operations

In [ ]:
add_op = a + b  # Could do tf.add(a, b), and [Shift]+[Tab] to look at the docs

In [ ]:
mult_op = a * b # Could do tf.multiply(a, b)

### Running Sessions to create Graphs with Feed Dictionaries

In [ ]:
with tf.Session() as sess:
    
    add_result = sess.run(add_op, feed_dict={a:10, b:20})
    print(add_result)
    
##endof:  with tf.Session() as sess:

In [ ]:
with tf.Session() as sess:
    
    add_result = sess.run(add_op, feed_dict={a:rand_a, b:rand_b})
    print(add_result)
    
    print('\n')
    
    mult_result = sess.run(mult_op, feed_dict={a:rand_a, b:rand_b})
    print(mult_result)
    
##endof:  with tf.Session as sess

Multiplication was element-by-element rather than a `dot` multiply.

<hr/>
<hr/>

## Example Neural Network

In [ ]:
n_features = 10
n_dense_neurons = 3

In [ ]:
# Placeholder for x
x = tf.placeholder(tf.float32,) #  shape is defined, though the 'None' is
                                #+ because it depends on how big of a 
                                #+ batch of data you're feeding into your NN
                                #+ feautures. We should be expecting 'x'
                                #+ to be receiving an array with n_samples
                                #+ by n_features 

In [ ]:
W = tf.Variable(tf.random_normal([n_features, n_dense_neurons]))

b = tf.Variable(tf.ones([n_dense_neurons])) # dimension must allow
                                            # matrix multiplication

<b>Operation Activation Function</b>

In [ ]:
xW = tf.matmul(x, W) # It will do the dot multiplication, now.

In [ ]:
z = tf.add(xW, b)

In [ ]:
a = tf.sigmoid(z)

<b>Variable Initializer!</b>

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    
    sess.run(init) # do this for a session
    
    layer_out = sess.run(a, feed_dict={x:np.random.random([1, n_features])})
                            # feeding it one sample
    
##endof:  with ... sess

In [ ]:
# print the result
print(layer_out) 
      #  Mine doesn't match his Jose's, even though I used
      #+ the same random seed and had the same results above
      #+ ... (?)
      #+ He says it might be different, depending on how many
      #+ times we had run random. I guess he ran random a few
      #+ times in between

We just passed in random numbers for `W` and `b`; we're not adjusting them.

We still need to finish off this process with optimization! Let's learn how to do this next.

<hr/>

## Full Network Example

Let's work on a regression example, we are trying to solve a very simple equation:

y = mx + b

y will be the y_labels and x is the x_data. We are trying to figure out the slope and the intercept for the line that best fits our data!

### Artificial Data (Some Made Up Regression Data)

a.k.a. (what the lecture calls it)

### Simple Regression Example

In [ ]:
x_data = np.linspace(0, 10, 10) + np.random.uniform(-1.5, 1.5, 10)

In [ ]:
x_data

In [ ]:
y_label = np.linspace(0, 10, 10) + np.random.uniform(-1.5, 1.5, 10)

In [ ]:
y_label

But that one matched (?). Maybe since it's np.random.uniform ...

In [ ]:
plt.plot(x_data, y_label, '*') #  What have we created?
                               #+ Hopefully something with
                               #+ a linear trend.

<b>Variables</b>

Remember, trying to solve $ y = mx + b $

In [ ]:
np.random.rand(2)

Just copying them in - 0.44 and 0.88. Wait, he wants us to use something different.

In [ ]:
m, b = np.random.rand(2)

In [ ]:
print("m = " + str(m) + " ; " + "b = " + str(b))

The optimizer needs tensors. Otherwise, trying to run
'optimizer.minimize(error)' below will give:

```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-40-f5a4ebeb180e> in <module>()
      1 optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
----> 2 train = optimizer.minimize(error)

~\.conda\envs\tfdeeplearning\lib\site-packages\tensorflow\python\training\optimizer.py in \\
minimize(self, loss, global_step, var_list, gate_gradients, aggregation_method, \\
         colocate_gradients_with_ops, name, grad_loss)
    398         aggregation_method=aggregation_method,
    399         colocate_gradients_with_ops=colocate_gradients_with_ops,
--> 400         grad_loss=grad_loss)
    401 
    402     vars_with_grad = [v for g, v in grads_and_vars if g is not None]

~\.conda\envs\tfdeeplearning\lib\site-packages\tensorflow\python\training\optimizer.py in \\
compute_gradients(self, loss, var_list, gate_gradients, aggregation_method, \\
                  colocate_gradients_with_ops, grad_loss)
    492                        "Optimizer.GATE_OP, Optimizer.GATE_GRAPH.  Not %s" %
    493                        gate_gradients)
--> 494     self._assert_valid_dtypes([loss])
    495     if grad_loss is not None:
    496       self._assert_valid_dtypes([grad_loss])

~\.conda\envs\tfdeeplearning\lib\site-packages\tensorflow\python\training\optimizer.py in \\
_assert_valid_dtypes(self, tensors)
    870     valid_dtypes = self._valid_dtypes()
    871     for t in tensors:
--> 872       dtype = t.dtype.base_dtype
    873       if dtype not in valid_dtypes:
    874         raise ValueError(

AttributeError: 'numpy.dtype' object has no attribute 'base_dtype'
```

That's why I can't _just_ pull them out of the array, like I did. I'll cast them.

In [ ]:
m = tf.Variable(m)
b = tf.Variable(b)
  # It's pythonic to repeat the variable, but I don't like it.

### Cost Function

In [ ]:
error = 0

In [ ]:
# zip will do tuples with points

for x, y in zip(x_data, y_label):
    y_hat = m*x + b # predicted y, m and b are tries
    
    error += (y - y_hat) ** 2
##endof:  for x, y ...

### Optimizer

Learning rate: we don't want to overshoot the value, nor do we want to take forever to train things. Computations are expensive, and that matters with networks, because time matters.

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(error)

### Initialize Variables

In [ ]:
init = tf.global_variables_initializer()

### Create Session and Run!

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    
    training_steps = 1 # probably way too low
    
    for i in range(training_steps):
        
        sess.run(train)
        
    ##endof:  for i ...

    final_slope, final_intercept = sess.run([m, b]) # expecting a very bad job
    
#endof:  with ... sess

In [ ]:
final_slope

In [ ]:
final_intercept

### Evaluate Results

In [ ]:
x_test = np.linspace(-1, 11, 10) # Make plot look a little nicer

# y = mx + b
y_pred_plot = final_slope * x_test + final_intercept

plt.plot(x_test, y_pred_plot, 'r')

plt.plot(x_data, y_label, '*')

It did decently with one epoch (one step). Let's go with 100.

### Do it with 100 epochs

In [ ]:
with tf.Session() as sess_100:
    
    sess_100.run(init)
    
    #training_steps = 1 # 100 will be better
    epochs = 100
    
    for i in range(epochs):
        
        sess_100.run(train)
        
    ##endof:  for i ...

    # Fetch the results
    final_slope_100, final_intercept_100 = sess_100.run([m, b])
    
#endof:  with ... sess_100

In [ ]:
final_slope_100

In [ ]:
final_intercept_100

### Evaluate with 100 epochs

In [ ]:
x_test = np.linspace(-1, 11, 10) # Make plot look a little nicer

# y = mx + b
y_pred_plot_100 = final_slope_100 * x_test + final_intercept_100

plt.plot(x_test, y_pred_plot_100, 'r')

plt.plot(x_data, y_label, '*')

<b>That does indeed seem better</b>

_That's all for now!_